In [ ]:
!pip install tensorflow

In [ ]:
!pip install PIL

In [ ]:
!pip install scipy

In [32]:
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [33]:
# Parameter
IMG_SIZE = (224, 224)  # Ukuran gambar untuk MobileNetV2
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.001

In [34]:
# Direktori dataset
train_dir = "train"
test_dir = "test"

In [35]:
# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [36]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [37]:
# Generator Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 6881 images belonging to 116 classes.


In [38]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 1292 images belonging to 116 classes.


In [39]:
# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

In [40]:
# Model Transfer Learning
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Bekukan layer pada model dasar

In [41]:
# Tambahkan lapisan khusus
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)  # Dropout untuk mencegah overfitting
x = Dense(256, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

In [42]:
# Buat model
model = Model(inputs=base_model.input, outputs=predictions)

In [43]:
# Kompilasi model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [44]:
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6)
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[reduce_lr, early_stopping]
)

Epoch 1/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 233s 2s/step - accuracy: 0.1474 - loss: 3.9455 - val_accuracy: 0.4861 - val_loss: 1.9212 - learning_rate: 0.0010
Epoch 2/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 234s 2s/step - accuracy: 0.4757 - loss: 1.8771 - val_accuracy: 0.6099 - val_loss: 1.3276 - learning_rate: 0.0010
Epoch 3/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 228s 2s/step - accuracy: 0.5801 - loss: 1.4211 - val_accuracy: 0.6920 - val_loss: 1.0753 - learning_rate: 0.0010
Epoch 4/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.6267 - loss: 1.2524 - val_accuracy: 0.7159 - val_loss: 0.9811 - learning_rate: 0.0010
Epoch 5/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 224s 2s/step - accuracy: 0.6639 - loss: 1.1250 - val_accuracy: 0.7314 - val_loss: 0.9290 - learning_rate: 0.0010
Epoch 6/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.6808 - loss: 1.0439 - val_accuracy: 0.7461 - val_loss: 0.8484 - learning_rate: 0.0010
Epoch 7/10
108/108 ━━━━━━━━━━━━━━━━━━━━ 225s 2s/step - accuracy: 0.7143 - loss: 0.